#### Notebook Summary as given by ChatGPT

- Presents a Number Type Theory that classifies all nonnegative integers by structural properties of their factorizations.
- Defines a total factorization function for n ≥ 0 and its inverse (factor_product), with tests illustrating the Fundamental Theorem of Arithmetic.
- Builds a hierarchy of predicates forming the Integer Type Tree (ITT) and implements a classifier that maps each n to one of 12 disjoint OEIS leaf sequences.
- Generates core and derived sequences, prints samples, and provides frequency counters and simple grid visualizations.
- Includes an OEIS search helper for sequence identification and an appendix with a LaTeX/TikZ diagram of the ITT for reference.

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Peter Luschny - October 2025

##Avant-propos

When I'm asked for a quick, easy introduction to combinatorics, I often refer to Gian-Carlo Rota's 'Twelvefold Way' (popularized by Richard Stanley's *Enumerative Combinatorics*). The Twelvefold Way is a systematic classification of 12 related combinatorial problems, organized in a table with 12 entries based on restrictions on the number and the color of balls in urns.

However, when I am faced with a similar question targeting elementary number theory, I can't point to anything comparable. 
Writing something similar to the TW was one of my motivations for this notebook. It was obvious to choose the Fundamental Theorem of Arithmetic as the starting point. Here we introduce 11 predicates that describe natural numbers solely in terms of their prime factorization.

To achieve a systematic representation, we will ensure that these predicates are mutually orthogonal. Thereby, every natural number is an element of precisely one of the 12 classes, and the union of these classes will encompass all natural numbers. This representation can then be clearly displayed as a binary tree. Interestingly, of the 23 nodes of this tree, 10 were added to OEIS only in the last 10 years.

In this notebook, we introduce the concept of factorization for all integers $ n \geq 0 $ and will adopt a fundamentally different primary classification than the one commonly used in the OEIS.

<h1 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Number Type Theory
</h1>

#### A taxonomy of integers from structural properties derived from the prime factorization.

## Factorization

Prime factorization is only defined for integers $ n \geq 2 $. 

- 1 has no prime factorization because 1 is not divisible by any prime.
- 0 has no prime factorization because 0 is divisible by every prime, so the factorization is not uniquely defined.

Informally, they describe the case "divisible by none" (for 1) and the case "divisible by all" (for 0).
We find it convenient to use the phrase **"n has no prime factors"** to describe both cases, although the phrases above are more precise.
In our setup, a factorization is always a **finite and nonempty list**, the exact form of which we will describe below.

However, we do not want to treat the two special cases, 0 and 1 separately in all our definitions and theorems, or to exclude them altogether. In fact, it is simple to include them by defining their factorizations as a list of tuples, like in the general case, call this the **factorization of n** for all integers $n \geq 0$, and reserve the term **prime factorization** if we implicitly assume $n \geq 2$.

### Factorization Definition

In this notebook, $\mathbb{N}$ denotes the nonnegative numbers, and we call this set the *natural numbers*. 
We will consider a function with signature (in Python notation)

* f(n: int) -> list[tuple[int, int]]
* Input: an integer $n \in \mathbb{N}$.
* Output: a finite list of pairs of natural numbers.

So if $f(n)$ is the output, then
$$
f(n) = \big[ (a_1, b_1), (a_2, b_2), \dots, (a_k, b_k) \big],
$$
for some $k \ge 0$, with each $(a_i, b_i) \in \mathbb{N} \times \mathbb{N}$ and $a_1 < a_2 < \dots < a_k$.

Since $k$ depends on $n$, the codomain is the **set of all finite sequences** over $\mathbb{N} \times \mathbb{N}$:
$$
(\mathbb{N} \times \mathbb{N})^* := \bigcup_{k \in \mathbb{N}} (\mathbb{N} \times \mathbb{N})^k.
$$
The notation $X^*$ (Kleene star) denotes the set of all finite sequences over $X$. Thus we have formally:
$$
f : \mathbb{N} \to (\mathbb{N} \times \mathbb{N})^*
$$
$$
f(n) \in (\mathbb{N} \times \mathbb{N})^* \quad\text{for all } n \in \mathbb{N}.
$$
This can be formulated equivalently: "$f$ is a function that assigns to each natural number $n$ a finite sequence of pairs of natural numbers that is monotone in its first variable." 

Such a function is the function **factorization**.


In [13]:
def factorization(n: int) -> list[tuple[int, int]]:
    """
    Return list of (p, e) for n >= 0 in increasing p.
    """
    # Special case: 0 and 1 are represented as [(n, 0)]
    if n < 2: return [(n, 0)]

    x = n          # Working copy of n to factor
    fact = []      # List to store (prime, exponent) pairs
    d = 2          # Start with smallest prime candidate

    # Trial division up to sqrt(x)
    while d * d <= x:
        if x % d == 0:  # d is a factor
            e = 0       # Count exponent of this prime factor
            while x % d == 0:  # Extract all powers of d
                x //= d
                e += 1
            fact.append((d, e))  # Store (prime, exponent)
        # Increment: first 2 to 3, then by 2s (skip even numbers)
        d += 1 if d == 2 else 2
    
    # If x > 1 after trial division, it's a remaining prime factor
    if x > 1:
        fact.append((x, 1))

    return fact

Note that 'factorization' always returns a nonempty list of pairs of natural numbers. 
Thus if 
<pre>
* factorization(n) = [(f1, e1), (f2, e2), ..., (fk, ek)], 
</pre>

then $k \geq 1$ for all $n \geq 0$. The factors are *strictly ordered*, i.e. $f_{j} < f_{j+1}$ for all $j$.

Since the factorization is a nonempty list also the two **projections**, 

<pre>
* factors(n)   := [f for (f, _) in factorization(n)] and 
* exponents(n) := [e for (_, e) in factorization(n)]
</pre>

are nonempty lists. No conventions are needed regarding the product, sum, or other operations in the case of an empty set to define the factorization of all natural numbers. 

In [14]:
for n in range(11): print(n, '→', factorization(n))

0 → [(0, 0)]
1 → [(1, 0)]
2 → [(2, 1)]
3 → [(3, 1)]
4 → [(2, 2)]
5 → [(5, 1)]
6 → [(2, 1), (3, 1)]
7 → [(7, 1)]
8 → [(2, 3)]
9 → [(3, 2)]
10 → [(2, 1), (5, 1)]


At this point, we need to take a closer look at the cases n = 0 and n = 1. In connection with the multiple applications of multiplication, the natural numbers fall into two classes: the numbers in one class are immune to any change if we multiply them by themself. In contrast the others change with each multiplication. For example, 
<pre>
* [0^n | n >= 1] = 0, 0, 0,  0,  0,  0,   0, ...
* [1^n | n >= 1] = 1, 1, 1,  1,  1,  1,   1, ...
* [2^n | n >= 1] = 2, 4, 8, 16, 32, 64, 128, ....
</pre>
Because of this remarkable property of 0 and 1, these numbers are called the *idempotent numbers*.
The exponent of these numbers (and only of these numbers) in the factorization is 0; all other factors have an exponent that is at least 1.

### Inverse of Factorization

Since each number has a unique factorization (which of course needs to be proven), we can consider its inverse function.

In [15]:
from math import prod

def factor_product(factors: list[tuple[int, int]]) -> int:
    """ Returns the product of the factors. Uses the
    modified power function: f^m if m >= 2, otherwise f."""
    return prod(f**m if m > 1 else f for (f, m) in factors)

The factor product is based on the *multiplicity function* 
$$ \mathbb{N} \times \mathbb{N} \rightarrow \mathbb{N}, \ (f, m) \rightarrow f^m \ \text{ if } m \ge 2, \text{otherwise } f. $$
The multiplicity function treats idempotent numbers separately, assigning them to themselves as their own powers. This function belongs to the family of the power function and is context-sensitive. One should not assume that this definition can be applied automatically to another subject area; for example, one usually chooses a different variant in combinatorics. 

The function *factor_product* reconstructs the integer from its factorization. It is the **inverse** of the factorization function, meaning that **factor_product(factorization(n)) = n** for all $n \geq 0$. In mathematics, this identity is called the **Fundamental Theorem of Arithmetic**. What makes this theorem fundamental are two facts:
- One fact is that for $n \geq 2$, the first component of every tuple in every list is a prime, and eventually all primes will appear this way. Note that we have not presupposed the notion of a 'prime' in the definition of the factorization or its inverse, so this is a non-trivial fact. 

We can use this fact to *define* the notion of a prime: An integer $p$ is prime if and only if its factorization is [(p, 1)]. We adopt this as our definition of a prime number.
- Another fact, and this is where the depth of the theorem resides, is that the prime factorization is **unique**. This means that no other list of tuples with primes in the first component that is ordered in lexicographical order will reconstruct the original integer.
 
The representation given here is valid for all $ n \geq 0 $ and smoothly integrates the cases $ n = 0 $ and $ n = 1 $. We do *not* use the convention for products over empty sets; instead, we rely on explicit definitions.

In [16]:
# When executing this test function you will only see some output if the test fails.

def FTA_test(lng: int) -> None:
    """Test the FTA (Fundamental Theorem of Arithmetic)"""
    for n in range(lng):
        f = factorization(n)
        p = factor_product(f)
        # Raise an assertion error if not equal
        assert n == p, "n={} f={} p={}".format(n, f, p)

FTA_test(100)

Some CAS systems completely exclude the case 0 and generate an error halt; others return an empty list for the cases $ n = 0 $ and $ n = 1 $, so these cases cannot be distinguished by their return value and must be handled explicitly.

### Predicates for Integer Types

We can classify integers according to structural properties derived from their  factorization. 
We have already seen two examples: **idempotent numbers** are numbers whose factorization is [(n, 0)];
**prime numbers** are numbers whose factorization is [(n, 1)]. We can define many more integer types in this way. For example, we define the type of **square numbers** as numbers whose prime factorization has only even exponents.

In principle, properties of integers should be defined directly by the properties of their factorization. We have a foundation (FTA), so let's build on it. 

For example, what is a composite number?
The popular website MathWorld says: "A composite number n is a positive integer n>1 which is not prime (i.e., which has factors other than 1 and itself)."
This is cumbersome and indirect. First, the concept of 'prime' is introduced, then it is negated, then 1 is excluded, and the case of 0 is not considered from the outset. This does not reflect conceptual thinking. A simple way to define a composite number is: A number is called **composite** if the sum of the exponents 
in its factorization is greater than or equal to 2. This covers everything, including the case of 0, and it satisfies our intuition: you need at least two things to make a composite number.

In the following, we have selected 11 predicates in this spirit based solely on factorization and will place them in a systematic context. Each predicate divides the set of integers into two disjoint subsets depending on whether the predicate applies to n or not.

<center>

| |Split | Predicate|
| -- | --| --|
| 1  | A → B ʌ C |  `max exps = 0`       | 
| 2  | C → D ʌ K |  `primes count = 1`   | 
| 3  | D → E ʌ H |  `max exps = 1`       | 
| 4  | K → L ʌ S |  `all exps equal`     | 
| 5  | E → F ʌ G |  `n ≡ 1 (mod 4)`         | 
| 6  | H → I ʌ J |  `max exps = 2`       | 
| 7  | L → M ʌ P |  `max exps = 1`       | 
| 8  | S → X ʌ T |  `gcd exps ≥ 2`       | 
| 9  | M → N ʌ O |  `primes count = 2`   | 
| 10 | P → Q ʌ R |  `max exps = 2`       | 
| 11 | T → U ʌ W |  `min exps > 1`       | 

</center>

By hierarchically combining these predicates, we obtain a tree-based classification of integers, the **Integer Type Tree (ITT)**. The inner nodes represent the splitting of the sets, while *the 12 leaves form a complete disjoint decomposition of the root* set, which is the set of the natural numbers including 0.

### An Integer Type Tree 
<center>
<img src="ITTree.png" width="720" height="500">
</center>

### The basic trisection

A typology such as the one above is, of course, not unique.
In the OEIS, the basic classification is usually described as follows.

The positive numbers 1, 2, ... are divided into three sets: 

- the unit {1}, 
- the primes, and 
- numbers with at least 2 factors > 1;
 
i.e. A000027 = {1} U A000040  U A002808.

In contrast, the top of our tree says:

The natural numbers 0, 1, 2, ... are divided into three sets: 
- the idempotent numbers {0, 1}, 
- the prime powers, and 
- numbers with at least 2 distinct prime factors; 

i.e. A001477 = A387487 U A246655 U A024619.

This alternative view emphasizes the special roles of 0 and 1 while highlighting their commonality, rather than their differences. While the third requirement in conventional classification demands the existence of at least two factors > 1, we require at least two distinct prime factors.

<center>
<img src="Idempotents.png" width="720" height="500">
</center>

### Illustrating the integration of the zero

An integer n is 'squarefree' if n is not divisible by a square greater than 1. Thus, 0 is not squarefree because 0 is divisible by 4, and 1 is squarefree, by special arrangement. Obviously, the definition of a squarefree number is convoluted.

MathWorld mentions "technical reasons" why Mathematica considers 1 to be squarefree, being a 'convention', and makes fun of it: "The number 1 therefore has the somewhat curious distinction of being simultaneously a perfect square and squarefree."

This is all confusing and contradicts mathematicians' ideal of building a consistent and beautiful theory. To illustrate how the integration of 0 works in our framework, we consider the definition of the notion 'kernel of a number' and the predicate 'is flat'. The 'kernel' of a number is the product of its distinct factors (multiplicities are ignored). If the kernel of n coincides with n, then we call n 'flat'. I.e., the flat numbers are the fixed points of the 'kernel' function. This is the simple formal statement, free of any quirks, and valid for all $n \geq 0$: 

In [17]:
from math import prod

# The kernel for n >= 0
def kernel(n: int) -> int:
    return prod(f for (f, _) in factorization(n))

def is_flat(n: int) -> bool: 
    return n == kernel(n)

The terms *is_flat* and *is_squarefree* are the same except for 0: *is_squarefree* excludes 0, while *is_flat* includes it. But that is not our view! 
For us, the 'real' squarefree numbers are A144338, that is, the 'nontrivial products of distinct primes'. And *is_flat* adds the idempotent numbers A387487 to these, because these are incapable of being raised to any real power, they are flat by nature.

In any case, 'n is flat' is a complete replacement for 'n is squarefree', provided $n \geq 2$. And since we only develop the right child of the root in our tree, we can do so safely in the following. Parenthetically, one could also use the word 'scenic' for 'nonflat'.

The definition of 'kernel' and 'is_flat' concerns the *first projection* of the factorization of n, binding them together by *multiplication*.
The functions 'exp_sum' and 'is_prime' are the counterparts using the *second projection* of the factorization of n, binding them together by *summation*.

In [18]:
# The sum of multiplicities, for n >= 0.
def exp_sum(n: int) -> int:
    return sum(e for (_, e) in factorization(n))

def is_prime(n: int) -> bool: 
    return 1 == exp_sum(n)

These two examples demonstrate how simply and consistently number-theoretic concepts can be defined for all $n \geq 0$. The fact that they are often defined only for $n \geq 1$ is sometimes defended by the remark that arithmetic functions, by definition, are only defined for $n \geq 1$. But this quickly becomes a circular justification.

As a supplement, we add the definition of the *Möbius function* μ(n) for $n \geq 0$.
For $n \in \{0, 1\}$ the definition below gives μ(0) = 1, so the term 0 will be prepended to A030229.

In [19]:
def moebius(n: int) -> int:
    if is_flat(n): return 1 if exp_sum(n) % 2 == 0 else -1
    return 0

print([moebius(n) for n in range(20)])
print([n for n in range(51) if moebius(n) == 1],  "A030229")
print([n for n in range(45) if moebius(n) == -1], "A030059") 
print([n for n in range(48) if moebius(n) == 0],  "A013929")
print([n for n in range(25) if moebius(n) <= 0],  "A390278") 
print([n for n in range(25) if moebius(n) >= 0],  "A174891")

[1, 1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1]
[0, 1, 6, 10, 14, 15, 21, 22, 26, 33, 34, 35, 38, 39, 46] A030229
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 30, 31, 37, 41, 42, 43] A030059
[4, 8, 9, 12, 16, 18, 20, 24, 25, 27, 28, 32, 36, 40, 44, 45] A013929
[2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24] A390278
[0, 1, 4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20, 21, 22, 24] A174891


Developing an alternative integer typology based on Moebius's trisection appears to be an appealing project. 


### Two dictionaries to describe the A-numbers and the nodes

Although we have fully described our program in the predicate table and the tree diagram above, we still need to convert it into a computer-readable representation. In addition to the names (which are the A-numbers of the OEIS and serve as the keys in the dictionary), we add a brief description of each sequence.

In [20]:
# A table with A-numbers and short descriptions
# Mapping of A-numbers to (parent A-number, description, node_label, predicate)
# Descriptions are short phrases, not complete sentences.

SEQUENCES: dict[str, tuple[str, str, str, str]] = { 
    "A001477": ("ROOT",    "Natural Numbers (nonnegative integers)", 'A', "n >= 0"),
    "A387487": ("A001477", "{0, 1}, idempotent numbers, no prime factor", 'B', "max exps = 0"),
    "A020725": ("A001477", "Has at least one prime factor, integers >= 2", 'C', "min exps >= 1"),
    "A246655": ("A020725", "All prime powers (p^k, k >= 1)", 'D', "# distinct primes = 1"),
    "A024619": ("A020725", "At least 2 distinct prime factors", 'K', "# distinct primes >= 2"),
    "A000040": ("A246655", "Primes, one factor with multiplicity = 1", 'E', "max exps = 1"),
    "A246547": ("A246655", "Proper prime powers (p^k, k >= 2)", 'H', "max exps >= 2"),
    "A182853": ("A024619", "Distinct factors > 1, all multiplicities = 1", 'L', "all exps = 1"),
    "A059404": ("A024619", "Different multiplicities", 'S', "exps not all equal"),
    "A120944": ("A182853", "Composite flat numbers", 'M', "flat composites"),
    "A303606": ("A182853", "Equi-powered composites, all multiplicities >= 2", 'P', "min exps >= 2"),
    "A177492": ("A303606", "Products of squares of 2 or more distinct primes", 'Q', "all exps = 2"),
    "A388304": ("A303606", "Prime factors > 1, all exps are equal and > 2", 'R', "all exps > 2 and equal"),
    "A303946": ("A059404", "Neither flat nor perfect powers", 'T', "gcd exps = 1"),
    "A389864": ("A059404", "Perfect powers with mixed multiplicity", 'X', "gcd exps >= 2"),
    "A002144": ("A000040", "Primes = 1 (mod 4) (Pythagorean primes)", 'F', "n ≡ 1 (mod 4)"),
    "A045326": ("A000040", "Primes = 3 (mod 4), or 2", 'G', "n ≡ 2,3 (mod 4)"),
    "A001248": ("A246547", "Squares of primes", 'I', "max exps = 2"),
    "A246549": ("A246547", "Higher prime powers (cubes, fourth powers, ...)", 'J', "max exps >= 3"),
    "A006881": ("A120944", "Product of two distinct primes", 'N', "# distinct primes = 2"),
    "A350352": ("A120944", "Products of >= 3 distinct primes", 'O', "# distinct primes >= 3"),
    "A052486": ("A303946", "Powerful but not perfect (Achilles numbers)", 'U', "min exps > 1"),
    "A332785": ("A303946", "Mixed prime powers: some multpl >= 2, some = 1", 'W', "min exps = 1"),
    "A144338": ("-",       "Products of distinct primes", '-', "-"),
    "A072777": ("-",       "Powers of products of distinct primes", '-', "-"),
    "A328956": ("-",       "sigma = omega * bigomega", '-', "-"),
}

# Helper functions to access the dictionary
def describe(anum: str) -> str:
    """Return short description for A-number or empty string if unknown."""
    val = SEQUENCES.get(anum)
    if val is not None:
        return val[1]  # description is the second element
    return "? not known"

def get_parent(anum: str) -> str:
    """Return parent A-number for given A-number."""
    val = SEQUENCES.get(anum)
    if val is not None:
        return val[0]  # parent is the first element
    return "? not known"

def get_node_label(anum: str) -> str:
    """Return node label for given A-number."""
    val = SEQUENCES.get(anum)
    if val is not None:
        return val[2]  # node label is the third element
    return "?"

def get_predicate(anum: str) -> str:
    """Return predicate string for given A-number (fourth tuple element)."""
    val = SEQUENCES.get(anum)
    if val is not None:
        return val[3]
    return "? not known"

In [21]:
# Node label to predicate mapping
NODE_PREDICATES: dict[str, str] = {
    'A': "n >= 0",
    'B': "max exps = 0",
    'C': "min exps >= 1",
    'D': "# distinct primes = 1",
    'E': "max exps = 1",
    'F': "n ≡ 1 (mod 4)",
    'G': "n ≡ 2,3 (mod 4)",
    'H': "max exps >= 2",
    'I': "max exps = 2",
    'J': "max exps >= 3",
    'K': "# distinct primes >= 2",
    'L': "all exps = 1",
    'M': "flat composites",
    'N': "# distinct primes = 2",
    'O': "# distinct primes >= 3",
    'P': "min exps >= 2",
    'Q': "all exps = 2",
    'R': "all exps > 2 and equal",
    'S': "exps not all equal",
    'T': "gcd exps = 1",
    'U': "min exps > 1",
    'W': "min exps = 1",
    'X': "gcd exps >= 2",
}

# Optional helper
def get_predicate_by_label(label: str) -> str:
    return NODE_PREDICATES.get(label, "? not known")

## The Classifier

The following function is the main function and could also be called the tree builder for the tree shown above. It assigns each integer $n \geq 0$ to one of 12 mutually disjoint classes whose union is $\mathbb{N}$. Apart from the functions 'factorization' and 'gcd' it does not use any external functions.

In [22]:
from math import gcd
from functools import reduce

def classifier(n: int) -> str:
    """
    Return the A-number of the sequence of which the given
    nonnegative integer n is a term (leaf in the tree).
    """

    # idempotents, n has no prime factors
    if n in (0, 1):
        return "A387487"  # idempotents 0 and 1

    # now prime factorization applies, i.e. n >= 2
    factors = factorization(n)  # list of (prime, exponent)

    primes_count = len(factors)
    exps = [e for (_, e) in factors]
    max_e = max(exps)
    min_e = min(exps)

    # n has at least one prime factor
    if primes_count == 1:
        if max_e == 1:
            return "A002144" if n % 4 == 1 else "A045326"
        else: # prime powers
            return "A001248" if max_e == 2 else "A246549"

    all_exps_equal = (len(set(exps)) == 1)
    # composite, multi-prime
    if all_exps_equal:
        if max_e == 1: # flat composites
            return "A006881" if primes_count == 2 else "A350352"
        else: # all exponents equal and ≥ 2
            return "A177492" if max_e == 2 else "A388304"

    # varied exponents
    g = reduce(gcd, exps)
    if g >= 2:
        return "A389864"
    else:
        return "A052486" if min_e > 1 else "A332785"

    # return "This can never happen!"  # sanity check

In [23]:
# Demonstration of the classifier

def classifier_demo(lng): 
    for n in range(lng):
        anum = classifier(n)
        node = get_node_label(anum)
        desc = describe(anum)
        print(f"{n:2d} → {anum} [{node}]  {desc}")

classifier_demo(33)

 0 → A387487 [B]  {0, 1}, idempotent numbers, no prime factor
 1 → A387487 [B]  {0, 1}, idempotent numbers, no prime factor
 2 → A045326 [G]  Primes = 3 (mod 4), or 2
 3 → A045326 [G]  Primes = 3 (mod 4), or 2
 4 → A001248 [I]  Squares of primes
 5 → A002144 [F]  Primes = 1 (mod 4) (Pythagorean primes)
 6 → A006881 [N]  Product of two distinct primes
 7 → A045326 [G]  Primes = 3 (mod 4), or 2
 8 → A246549 [J]  Higher prime powers (cubes, fourth powers, ...)
 9 → A001248 [I]  Squares of primes
10 → A006881 [N]  Product of two distinct primes
11 → A045326 [G]  Primes = 3 (mod 4), or 2
12 → A332785 [W]  Mixed prime powers: some multpl >= 2, some = 1
13 → A002144 [F]  Primes = 1 (mod 4) (Pythagorean primes)
14 → A006881 [N]  Product of two distinct primes
15 → A006881 [N]  Product of two distinct primes
16 → A246549 [J]  Higher prime powers (cubes, fourth powers, ...)
17 → A002144 [F]  Primes = 1 (mod 4) (Pythagorean primes)
18 → A332785 [W]  Mixed prime powers: some multpl >= 2, some = 1


## Generating sequences

In [24]:
# Generate all tree leaves quickly in one pass

Anames = ['A387487', 'A002144', 'A045326', 'A001248', 'A246549', 'A006881', 
          'A350352', 'A389864', 'A332785', 'A052486', 'A388304', 'A177492']
Aseq = {aname: [] for aname in Anames}

def show_classification(search_length: int, display_length: int) -> None:
    for n in range(search_length):
        anum = classifier(n)
        if anum in Aseq:
            Aseq[anum].append(n)
        else:
            print("UHUH...", n, anum)

    print("*** The leaves of the type tree are:\n")
    for anum in Anames:
        print(anum, [get_node_label(anum)], describe(anum), '\n', 
              Aseq[anum][:display_length])

show_classification(11000, 20)

*** The leaves of the type tree are:

A387487 ['B'] {0, 1}, idempotent numbers, no prime factor 
 [0, 1]
A002144 ['F'] Primes = 1 (mod 4) (Pythagorean primes) 
 [5, 13, 17, 29, 37, 41, 53, 61, 73, 89, 97, 101, 109, 113, 137, 149, 157, 173, 181, 193]
A045326 ['G'] Primes = 3 (mod 4), or 2 
 [2, 3, 7, 11, 19, 23, 31, 43, 47, 59, 67, 71, 79, 83, 103, 107, 127, 131, 139, 151]
A001248 ['I'] Squares of primes 
 [4, 9, 25, 49, 121, 169, 289, 361, 529, 841, 961, 1369, 1681, 1849, 2209, 2809, 3481, 3721, 4489, 5041]
A246549 ['J'] Higher prime powers (cubes, fourth powers, ...) 
 [8, 16, 27, 32, 64, 81, 125, 128, 243, 256, 343, 512, 625, 729, 1024, 1331, 2048, 2187, 2197, 2401]
A006881 ['N'] Product of two distinct primes 
 [6, 10, 14, 15, 21, 22, 26, 33, 34, 35, 38, 39, 46, 51, 55, 57, 58, 62, 65, 69]
A350352 ['O'] Products of >= 3 distinct primes 
 [30, 42, 66, 70, 78, 102, 105, 110, 114, 130, 138, 154, 165, 170, 174, 182, 186, 190, 195, 210]
A389864 ['X'] Perfect powers with mixed multiplicit